In [2]:
# -*- encoding:urf-8 -*-
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table like and matrices
import pandas as pd
import numpy as np

# Modeling Helper
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import missingno as msno

# Configure visualization
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 10,8
poi_path = '../../UAI_data/input/poi.csv'
poi_raw_path = '../../UAI_data/input/poi_raw.csv'
train_Aug_path = '../../UAI_data/input/train_Aug.csv'
train_Jul_path = '../../UAI_data/input/train_July.csv'
weather_path = '../../UAI_data/input/weather.csv'
test_pub5k_path = '../../UAI_data/input/test_id_Aug_agg_public5k.csv'
test_path = '../../UAI_data/input/UAI_sample_submission.csv'

poi = pd.read_csv(poi_path,encoding='gbk')
poi_raw = pd.read_csv(poi_raw_path,encoding='gbk')
train_aug = pd.read_csv(train_Aug_path,encoding='gbk')
train_jul = pd.read_csv(train_Jul_path,encoding='gbk')
weather = pd.read_csv(weather_path)
test_pub5k = pd.read_csv(test_pub5k_path,encoding='gbk')

### Re-encode location

In [69]:
l_7_end_geo = train_jul['end_geo_id']
l_7_start_geo = train_jul['start_geo_id']
l_8_end_geo = train_aug['end_geo_id']
l_8_start_geo = train_aug['start_geo_id']
l_poi_geo = poi_raw['location']
l_start_test = test_pub5k['start_geo_id']
l_end_test = test_pub5k['end_geo_id']

l_location = pd.concat([l_7_end_geo,l_7_start_geo,l_8_start_geo,l_8_end_geo,l_start_test,l_end_test],axis=0)
l_location = l_location.unique()
l_location_id = np.arange(l_location.shape[0])
data = {'location':l_location,'location_id':l_location_id}
location = pd.DataFrame(data)
location.to_csv('../../UAI_data/input/location.csv',index=False)
# l_start_geo
print "Jul train columns", train_jul.columns
print "Aug train columns", train_aug.columns
print 'test columns',test_pub5k.columns
print 'poi columns', poi_re.columns

Jul train columns Index([u'id', u'driver_id', u'member_id', u'create_date', u'create_hour',
       u'status', u'estimate_money', u'estimate_distance', u'estimate_term',
       u'start_geo_id', u'end_geo_id'],
      dtype='object')
Aug train columns Index([u'id', u'driver_id', u'member_id', u'create_date', u'create_hour',
       u'status', u'estimate_money', u'estimate_distance', u'estimate_term',
       u'start_geo_id', u'end_geo_id'],
      dtype='object')
test columns Index([u'test_id', u'start_geo_id', u'end_geo_id', u'create_date',
       u'create_hour'],
      dtype='object')
poi columns Index([    u'gas_station_num',     u'supermarket_num',       u'residence_num',
                 u'metro_num',     u'bus_station_num',           u'caffe_num',
            u'restaurant_num',             u'ATM_num', u'office_building_num',
                 u'hotel_num',         u'location_id'],
      dtype='object')


### Re-encode id

In [70]:
l_7_id= train_jul['id']
l_8_id = train_aug['id']

l_id = pd.concat([l_7_id,l_8_id],axis=0)
l_id = l_id.unique()
l_id_new = np.arange(l_id.shape[0])
data = {'id':l_id,'id_new':l_id_new}
ID = pd.DataFrame(data)
ID.to_csv('../../UAI_data/input/id.csv',index=False)

### Re-encode member

In [71]:
l_7_member_id= train_jul['member_id']
l_8_member_id = train_aug['member_id']

l_member_id = pd.concat([l_7_member_id,l_8_member_id],axis=0)
l_member_id = l_member_id.unique()
l_member_id_new = np.arange(l_member_id.shape[0])
data = {'member_id':l_member_id,'member_id_new':l_member_id_new}
member = pd.DataFrame(data)
member.to_csv('../../UAI_data/input/member_id.csv',index=False)

### Re-encode driver

In [72]:
l_7_driver_id= train_jul['driver_id']
l_8_driver_id = train_aug['driver_id']

l_driver_id = pd.concat([l_7_driver_id,l_8_driver_id],axis=0)
l_driver_id = l_driver_id.unique()
l_driver_id_new = np.arange(l_driver_id.shape[0])
data = {'driver_id':l_driver_id,'driver_id_new':l_driver_id_new}
driver = pd.DataFrame(data)
driver.to_csv('../../UAI_data/input/driver_id.csv',index=False)

### train_Jul reshape

In [73]:
location = pd.read_csv('../input/location.csv')
loc_cp = location.copy()

#driver id 
driver = pd.read_csv('../input/driver_id.csv')
train_jul = pd.merge(train_jul,driver,on='driver_id',how='left')
train_jul.drop(['driver_id'],axis=1,inplace = True)

# member_id
member = pd.read_csv('../input/member_id.csv')
train_jul = pd.merge(train_jul,member,on='member_id',how='left')
train_jul.drop(['member_id'],axis=1,inplace = True)

#start_geo
loc_cp.rename(columns={'location':'start_geo_id'},inplace=True)
loc_cp.rename(columns={'location_id':'start_geo_id_new'},inplace=True)
train_jul = pd.merge(train_jul,loc_cp,on='start_geo_id',how='left')
train_jul.drop(['start_geo_id'],axis=1,inplace = True)

#end_geo
loc_cp = location.copy()
loc_cp.rename(columns={'location':'end_geo_id'},inplace=True)
loc_cp.rename(columns={'location_id':'end_geo_id_new'},inplace=True)
train_jul = pd.merge(train_jul,loc_cp,on='end_geo_id',how='left')
train_jul.drop(['end_geo_id'],axis=1,inplace = True)

#id
ID = pd.read_csv('../input/id.csv')
train_jul = pd.merge(train_jul,ID,on='id',how='left')
train_jul.drop(['id'],axis=1,inplace = True)

train_jul.rename(columns={'id_new':'id',
                         'driver_id_new':'driver_id',
                         'member_id_new':'member_id',
                         'start_geo_id_new':'start_geo_id',
                         'end_geo_id_new':'end_geo_id'
                        },inplace=True)
train_jul.to_csv('../../UAI_data/input/train_July_re.csv',encoding='gbk',index=False)
train_jul

,create_date,create_hour,status,estimate_money,estimate_distance,estimate_term,driver_id,member_id,start_geo_id,end_geo_id,id
0,2017-07-01,0,2,140.00,20099.0,18.0,0,0,5,0,0
1,2017-07-01,0,2,78.00,9000.0,18.0,1,1,22,1,1
2,2017-07-01,0,1,86.23,10323.0,20.0,2,2,39,2,2
3,2017-07-01,0,1,81.88,14197.0,27.0,3,3,55,3,3
4,2017-07-01,2,1,217.01,39150.0,40.0,2,4,195,4,4
5,2017-07-01,2,1,55.69,8360.0,16.0,4,5,22,1,5
6,2017-07-01,6,2,188.00,40184.0,47.0,5,6,206,5,6
7,2017-07-01,8,2,56.00,10953.0,21.0,6,7,55,6,7
8,2017-07-01,9,2,25.00,2889.0,5.0,7,8,68,7,8
9,2017-07-01,9,2,31.00,4415.0,8.0,8,9,63,8,9


### train_Aug reshape

In [74]:
location = pd.read_csv('../input/location.csv')
loc_cp = location.copy()

#driver id 
driver = pd.read_csv('../input/driver_id.csv')
train_aug = pd.merge(train_aug,driver,on='driver_id',how='left')
train_aug.drop(['driver_id'],axis=1,inplace = True)

# member_id
member = pd.read_csv('../input/member_id.csv')
train_aug = pd.merge(train_aug,member,on='member_id',how='left')
train_aug.drop(['member_id'],axis=1,inplace = True)

#start_geo
loc_cp.rename(columns={'location':'start_geo_id'},inplace=True)
loc_cp.rename(columns={'location_id':'start_geo_id_new'},inplace=True)
train_aug = pd.merge(train_aug,loc_cp,on='start_geo_id',how='left')
train_aug.drop(['start_geo_id'],axis=1,inplace = True)

#end_geo
loc_cp = location.copy()
loc_cp.rename(columns={'location':'end_geo_id'},inplace=True)
loc_cp.rename(columns={'location_id':'end_geo_id_new'},inplace=True)
train_aug = pd.merge(train_aug,loc_cp,on='end_geo_id',how='left')
train_aug.drop(['end_geo_id'],axis=1,inplace = True)

#id
ID = pd.read_csv('../input/id.csv')
train_aug = pd.merge(train_aug,ID,on='id',how='left')
train_aug.drop(['id'],axis=1,inplace = True)

train_aug.rename(columns={'id_new':'id',
                         'driver_id_new':'driver_id',
                         'member_id_new':'member_id',
                         'start_geo_id_new':'start_geo_id',
                         'end_geo_id_new':'end_geo_id'
                        },inplace=True)
train_aug.to_csv('../../UAI_data/input/train_Aug_re.csv',encoding='gbk',index=False)
train_aug

,create_date,create_hour,status,estimate_money,estimate_distance,estimate_term,driver_id,member_id,start_geo_id,end_geo_id,id
0,2017-08-01,12,2,42.00,7324.0,14.0,2960,36797,61,85,892514
1,2017-08-01,12,2,37.00,5901.0,11.0,1564,62541,6,16,892515
2,2017-08-01,14,2,32.00,3137.0,6.0,592,85350,24,24,892516
3,2017-08-01,14,0,34.53,3672.0,7.0,2,184371,70,70,892517
4,2017-08-01,16,2,31.00,4392.0,8.0,2198,132061,46,41,892518
5,2017-08-01,16,2,61.00,9645.0,19.0,3025,184372,42,167,892519
6,2017-08-01,18,2,39.00,6514.0,13.0,970,184373,34,41,892520
7,2017-08-01,20,0,142.76,19667.0,39.0,2,184374,47,1,892521
8,2017-08-01,20,2,50.00,7228.0,14.0,3180,23361,16,6,892522
9,2017-08-01,20,2,78.00,16359.0,32.0,392,184375,18,133,892523


### poi_re reshape

In [75]:
location = pd.read_csv('../input/location.csv')
loc_cp = location.copy()
poi_raw = pd.read_csv(poi_raw_path,encoding='gbk')
poi_re = pd.merge(poi_raw,loc_cp, on='location',how='left')
del poi_re['location'],poi_re['gas_station'],poi_re['supermarket'],poi_re['residence'],poi_re['metro'],poi_re['bus_station'],poi_re['caffe'],poi_re['restaurant'],poi_re['ATM'],poi_re['office_building'],poi_re['hotel']
poi_re.to_csv('../../UAI_data/input/poi_re.csv',index=False)
poi_re

,gas_station_num,supermarket_num,residence_num,metro_num,bus_station_num,caffe_num,restaurant_num,ATM_num,office_building_num,hotel_num,location_id
0,0,0,1,0,2,0,0,0,0,0,220
1,4,43,151,4,36,22,597,59,47,45,79
2,0,0,0,0,0,0,0,0,0,0,897
3,4,32,50,3,22,6,509,20,9,27,107
4,0,3,1,0,6,0,6,0,0,1,624
5,0,0,0,0,0,0,0,0,0,0,763
6,2,30,151,3,28,19,513,39,55,80,33
7,0,0,0,0,0,0,0,0,0,0,869
8,9,34,170,4,45,20,779,69,66,116,3
9,0,0,0,0,0,0,0,0,0,0,803


### test reshape

In [76]:
location = pd.read_csv('../input/location.csv')
#start_geo
loc_cp = location.copy()
loc_cp.rename(columns={'location':'start_geo_id'},inplace=True)
loc_cp.rename(columns={'location_id':'start_geo_id_new'},inplace=True)
test_pub5k = pd.merge(test_pub5k,loc_cp,on='start_geo_id',how='left')
test_pub5k.drop(['start_geo_id'],axis=1,inplace = True)

#end_geo
loc_cp = location.copy()
loc_cp.rename(columns={'location':'end_geo_id'},inplace=True)
loc_cp.rename(columns={'location_id':'end_geo_id_new'},inplace=True)
test_pub5k = pd.merge(test_pub5k,loc_cp,on='end_geo_id',how='left')
test_pub5k.drop(['end_geo_id'],axis=1,inplace = True)
test_pub5k.to_csv('../../UAI_data/input/test.csv',index=False)
test_pub5k

,test_id,create_date,create_hour,start_geo_id_new,end_geo_id_new
0,0,2017-08-01,21,24,27
1,1,2017-08-01,21,24,9
2,2,2017-08-01,21,24,22
3,3,2017-08-01,17,24,9
4,4,2017-08-01,17,55,55
5,5,2017-08-01,17,110,88
6,6,2017-08-01,9,23,21
7,7,2017-08-01,21,16,50
8,8,2017-08-01,13,6,24
9,9,2017-08-01,21,6,22
